# Mass modelling

Read stellar surface density profiles and fit:
- the angular diameter distance $D_A$
- the parameters of the dark matter halo

# 1. Initialisation

External libraries

In [ ]:
#%matplotlib ipympl
import os
import glob
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import AutoMinorLocator
from astropy.table import QTable
from astropy import units as u
from astropy import constants as c

In [ ]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
def new_figure(fig_name, figsize=(6, 4), nrows=1, ncols=1, sharex='col', sharey='row', gridspec_kw={'hspace': 0, 'wspace': 0}, suptitle=True):
    plt.close(fig_name)
    fig = plt.figure(fig_name, figsize=figsize, layout="constrained")
    axes = fig.subplots(nrows=nrows, ncols=ncols, squeeze=False,
                        sharex=sharex, sharey=sharey,
                        gridspec_kw=gridspec_kw
                       )
    #fig.set_tight_layout(True)
    for ax in axes.flat:
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')

    if suptitle is True:
        fig.suptitle(fig_name)
    elif suptitle is not False and suptitle is not None:
        fig.suptitle(suptitle)
    
    return fig, axes

## Directories

In [ ]:
def test_dir(dir_name):
    if not os.path.isdir(dir_name):
        print(f'>> WARNING: Creating directory "{dir_name}"')
        os.makedirs(dir_name)
    return(dir_name)

In [ ]:
input_dir = '.'
output_dir = test_dir('output')
maps_dir = test_dir(os.path.join(output_dir, 'maps'))
plots_dir = test_dir(os.path.join(output_dir, 'plots'))
profiles_dir = test_dir(os.path.join(output_dir, 'profiles'))
params_dir = test_dir(os.path.join(output_dir, 'parameters'))

# 2. Comparison to WALLABY

## WALLABY catalogue

In [ ]:
WALLABY_catalogue = QTable.read('AS102_Derived_Catalogue_wallaby_pilot_dr1_kinmodel_cat_v01_5844.csv')

In [ ]:
WALLABY_catalogue

## Comparison

In [ ]:
H0 = 72
reference_distance = u.Mpc

for profiles_filename in glob.glob(os.path.join(profiles_dir, "*"))[:]:
    target_name = os.path.basename(profiles_filename).split('_')[0]
    print(target_name)

    # WALLABY
    wallaby_row = WALLABY_catalogue[WALLABY_catalogue['name'] == target_name][0]
    wallaby_theta = np.array(wallaby_row['rad'].split(','), dtype=float) << u.arcsec
    wallaby_vrot = np.array(wallaby_row['vrot_model'].split(','), dtype=float) << u.km/u.s
    wallaby_distance = wallaby_row['vsys_model'] / H0 << u.Mpc
    wallaby_gravity = (wallaby_vrot**2 / (reference_distance * np.sin(wallaby_theta))).to(u.km/u.s**2)

    # encosed gas mass and gravity
    wallaby_theta_gas = np.array(wallaby_row['rad_sd'].split(','), dtype=float) << u.arcsec
    wallaby_gas = np.array(wallaby_row['sd_model'].split(','), dtype=float) << u.Msun/u.pc**2
    gas_theta_bins = np.linspace(0, wallaby_theta_gas[-1], 100)
    theta_mid = (gas_theta_bins[1:] +  gas_theta_bins[:-1]) / 2
    bin_area = np.pi * (gas_theta_bins[1:]**2 - gas_theta_bins[:-1]**2)
    gas_mass = np.cumsum(bin_area * np.interp(
        theta_mid.value, wallaby_theta_gas.value, wallaby_gas.value, left=wallaby_gas[0].value))
    gas_theta_bins = gas_theta_bins[1:]
    gas_gravity = (c.G * gas_mass * wallaby_gas.unit / gas_theta_bins**2).to(u.km/u.s**2)
    

    # radial density profile
    if not os.path.isfile(profiles_filename):
        continue
    profiles_table = QTable.read(profiles_filename)
    theta = profiles_table['theta'] << u.arcsec
    surface_density = profiles_table['median'] << u.Msun/u.pc**2
    radius = np.sin(theta) * reference_distance

    # inclination
    params_filename = os.path.join(params_dir, f"{target_name}_params.csv")
    if not os.path.isfile(params_filename):
        continue
    ra, dec, inclination, pa = np.loadtxt(params_filename)
    if np.isnan(inclination):
        continue
    inclination *= u.deg
    surface_density *= np.cos(inclination)

    # encosed mass and stellar gravity
    theta_bins = np.linspace(0, theta[-1], 100)
    theta_mid = (theta_bins[1:] +  theta_bins[:-1]) / 2
    bin_area = np.pi * (theta_bins[1:]**2 - theta_bins[:-1]**2)
    stars_mass = np.cumsum(bin_area * np.interp(
        theta_mid.value, theta.value, surface_density.value, left=surface_density[0].value))
    theta_bins = theta_bins[1:]
    stars_gravity = (c.G * stars_mass * surface_density.unit / theta_bins**2).to(u.km/u.s**2)
    stars_model = (np.interp(wallaby_theta, theta_bins, stars_gravity) * np.sin(wallaby_theta) * reference_distance).to(u.km**2/u.s**2)
    #stars_model /=  np.sum(vrot_model**2)
    baryons_model = stars_model + (np.interp(wallaby_theta, gas_theta_bins, gas_gravity) * np.sin(wallaby_theta) * reference_distance).to(u.km**2/u.s**2)

    # dark matter
    i_max = np.nanargmax(wallaby_vrot)
    theta_max = wallaby_theta[i_max]
    v_max = wallaby_vrot[i_max]
    candidate_a, candidate_v = np.meshgrid(np.linspace(theta_max, 5*theta_max, 100), np.linspace(.1*v_max, 3*v_max, 100))
    candidate_halo = candidate_v[:,:, np.newaxis] * np.sqrt(wallaby_theta[np.newaxis, np.newaxis, :] / theta_max)
    candidate_halo *= (candidate_a[:,:, np.newaxis] + theta_max) / (candidate_a[:,:, np.newaxis] + wallaby_theta[np.newaxis, np.newaxis, :])
    residual = (wallaby_vrot**2)[np.newaxis, np.newaxis, :] - candidate_halo**2
    weight = np.where(residual > 0, 1, 0)
    dynamic_distance = np.sum(weight * residual * baryons_model[np.newaxis, np.newaxis, :], axis=2)
    dynamic_distance /= np.sum(weight * (baryons_model**2)[np.newaxis, np.newaxis, :], axis=2)
    residual -= baryons_model[np.newaxis, np.newaxis, :] * dynamic_distance[:, :, np.newaxis]
    chi = np.sqrt(np.sqrt(np.nanmean(residual**2, axis=2)))
    if np.all(np.isnan(chi)):
        continue
    
    best = np.unravel_index(np.nanargmin(chi), chi.shape)
    dynamic_distance *= reference_distance
    best_distance = dynamic_distance[best]
    best_vrot = np.sqrt(stars_gravity*np.sin(theta_bins)*dynamic_distance[best]).to(u.km/u.s)
    best_halo = candidate_v[best] * np.sqrt(theta_bins / theta_max) * (candidate_a[best] + theta_max) / (candidate_a[best] + theta_bins)
    best_gas = np.sqrt(gas_gravity*np.sin(gas_theta_bins)*dynamic_distance[best]).to(u.km/u.s)
    best_total = np.sqrt(best_halo**2 + best_vrot**2 + np.interp(theta_bins, gas_theta_bins, best_gas)**2)
    
    np.savetxt(os.path.join(params_dir, f'{target_name}_distance.csv'),
               [wallaby_row['vsys_model'] / c.c.to_value(u.km/u.s),
                best_distance.to_value(u.Mpc),
                candidate_v[best].to_value(u.km/u.s),
                candidate_a[best].to_value(u.arcsec)],
               header='z, distance, v_max_halo, r_max_halo',
               fmt='%.4f')

    # Figures: distance fit
    fig_name = f'{target_name}_distance_fit'
    fig, axes = new_figure(fig_name)
    ax = axes[0, 0]

    im = ax.contourf(candidate_a.value, candidate_v.value, dynamic_distance.value, cmap='nipy_spectral')
    plt.colorbar(im)
    im = ax.contour(candidate_a.value, candidate_v.value, chi.value, colors='k')
    ax.plot(candidate_a[best], candidate_v[best], 'ko', label=f'V={candidate_v[best]:.2f}, a={candidate_a[best]:.2f}')

    ax.legend()
    ax.set_xlabel(r'$\theta_{\rm max}$ [arcsec]')
    ax.set_ylabel(r'$V_{\rm max}$ [km/s]')
    fig.savefig(os.path.join(plots_dir, f"{fig_name}.png"), facecolor='white')
    plt.close('all')


    # Figures: rotation curve
    fig_name = f'{target_name}_Vrot'
    fig, axes = new_figure(fig_name)
    ax = axes[0, 0]

    ax.plot(wallaby_theta, wallaby_vrot, 'ks', label=f'$v/H_0$ = {wallaby_distance:.2f}')
    ax.plot(theta_bins, best_total, 'k-', label=f'This work: D={dynamic_distance[best]:.2f}, chi={chi[best]:.2f}')
    ax.plot(theta_bins, best_halo, 'k:', label=f'halo (V={candidate_v[best]:.2f}, a={candidate_a[best]:.2f})')
    ax.plot(theta_bins, best_vrot, 'b--', label=f'stars')
    ax.plot(theta_bins, best_gas, 'y:', label=f'gas')
    
    ax.set_ylabel(r'rotation velocity [km/s]')
    ax.set_ylim(.5, 500)
    ax.set_yscale('log')
    ax.set_xlabel(r'galactocentric distance $\theta$ [arcsec]')
    ax.set_xlim(0, 240)
    ax.legend(loc='lower right')
    fig.savefig(os.path.join(plots_dir, f"{fig_name}.png"), facecolor='white')
    plt.close('all')
    
    continue
    
    # individual figures
    fig_name = f'{target_name}_Vrot'
    fig, axes = new_figure(fig_name)
    ax = axes[0, 0]

    for a in np.linspace(r2/2, r2*2, 100):
        halo = v2 * np.sqrt(theta_bins/r2) * (a + r2) / (a + theta_bins)
        ax.plot(theta_bins, halo, 'k-', alpha=.2, label=f'{a:.2f}')
        halo = v2 * np.sqrt(wallaby_theta/r2) * (a + r2) / (a + wallaby_theta)
        residual = wallaby_vrot**2 - halo**2
        weight = np.where(residual > 0, 1, 0)
        if np.count_nonzero(good) > 0:
            dynamic_distance = (np.sum(weight * residual * stars_model[np.newaxis, np.newaxis, :]) / np.sum(weight * stars_model[good]**2)).to(u.dimensionless_unscaled)
            ax.plot(wallaby_theta, np.sqrt(stars_model*dynamic_distance), 'b-', alpha=.2, label=f'{dynamic_distance:.2f}')
    '''
    ax.plot(wallaby_theta, wallaby_gravity, 'ks', label=f'WALLABY ({wallaby_distance:.2f})')
    ax.plot(theta_bins, wallaby_halo, 'b:', label=f'a = {a:.2f}')
    ax.plot(wallaby_theta, wallaby_stars*inverse_distance, 'b*', label=f'D = {dynamic_distance:.2f}')
    ax.plot(theta_bins, stars_gravity, 'k--', label=f'stars')
    '''
    ax.plot(wallaby_theta, wallaby_vrot, 'k+', label=f'WALLABY ({wallaby_distance:.2f})')
    ax.plot(theta_bins, np.sqrt(wallaby_halo*np.sin(theta_bins)*reference_distance).to(u.km/u.s), 'k:', label=f'DM halo ({wallaby_a:.2f})')
    #ax.plot(theta_bins, np.sqrt((hernquist_vrot*hernquist_factor)**2 + (vrot*vrot_factor)**2), 'k-', label=f'total ($\\chi^2 = ...)')

    #ax.set_ylabel(r'surface density $\Sigma$ [M$_\odot$/pc$^2$]')
    #ax.set_ylim(.03, 3e4)
    ax.set_ylim(.5, 500)
    ax.set_yscale('log')
    ax.legend()
    
    ax.set_xlabel(r'galactocentric distance $\theta$ [arcsec]')
    #ax.set_xlim(0, theta[2].value)

    fig.savefig(os.path.join(plots_dir, f"{fig_name}.png"), facecolor='white')
    #plt.close('all')


# 3. Distances

In [ ]:
z = []
DA = []
for distance_filename in glob.glob(os.path.join(params_dir, "*dist*csv"))[:]:
    target_name = os.path.basename(distance_filename).split('_')[0]
    print(target_name)
    
    zz, D = np.loadtxt(distance_filename)[:2]
    z.append(zz)
    DA.append(D)
z = np.array(z)
DA = np.array(DA)

In [ ]:
fig_name = f'Dynamical distance estimates'
fig, axes = new_figure(fig_name)
ax = axes[0, 0]

v = z * c.c.to(u.km/u.s)
ax.plot(v, DA, 'k.')
vv = np.linspace(0, np.nanmax(v), 100)
ax.plot(vv, vv / (72*u.km/u.s), 'k--', label='$H_0 = 72$ km/s/Mpc')

ax.set_ylabel('angular diameter distance [Mpc]')
#ax.set_ylim(0, 160)
ax.set_xlabel('recession velocity [km/s]')
ax.set_xlim(0)
ax.legend()
fig.savefig(os.path.join(output_dir, "H0.png"), facecolor='white')


In [ ]:
target_name

In [ ]:
z*c.c

# --- STOP

In [ ]:
raise BaseException("stop here")

# 3. Comparison to SPARC

In [ ]:
for galaxy in SPARC_catalogue[:]:
    # individual figures
    fig_name = f'{galaxy["Galaxy"]}_SPARC'
    fig, axes = new_figure(fig_name)
    ax = axes[0, 0]

    ax.plot(theta, surface_density, 'k-+', label=f'This work (i={inclination:.2f})')
    ax.plot(SPARC_theta, SPARC_stars, 'k:.', label=f'SPARC (i={median_i:.2f}, $\\Upsilon_{{bulge}}\sim${median_bul:.2f}, $\\Upsilon_{{disk}}\sim${median_disk:.2f})')
    ax.fill_between(comparison_theta.to_value(u.arcsec),
                   np.fmin(comparison_SPARC, comparison_us).to_value(u.Msun/u.pc**2),
                   np.fmax(comparison_SPARC, comparison_us).to_value(u.Msun/u.pc**2),
                   color='red', alpha=.2)
    ax.legend(title=f'{galaxy["Galaxy"]} (D={median_D:.2f})')
    ax.set_yscale('log')
    
    fig.savefig(os.path.join(plots_dir, f"{fig_name}.png"), facecolor='white')
    if not show_plots:
        plt.close(fig_name)

In [ ]:
x = (p50/comparison_surface_density_SPARC)[comparison_surface_density_SPARC > 10]
x, np.nanmean(x), np.nanstd(x), np.log10(np.nanmean(x)), np.nanmean(np.log10(p84/p16)[comparison_surface_density_SPARC > 10]/2)

In [ ]:
print(galaxy)
bins = np.where(SPARC_models['ID'] == galaxy['Galaxy'])
SPARC_models[bins]

In [ ]:
SPARC_theta

In [ ]:
SPARC_stars

In [ ]:
comparison_theta

In [ ]:
theta[:np.argmin(surface_density[1:])]

## Mass model

In [ ]:
for galaxy in SPARC_catalogue[100:101]:
    if np.ma.is_masked(galaxy['Galaxy']):
        continue

    params_filename = os.path.join(params_dir, f"{galaxy['Galaxy']}_params.csv")
    if not os.path.isfile(params_filename):
        continue
    ra, dec, inclination, pa = np.loadtxt(params_filename)
    inclination *= u.deg
    
    profiles_filename = os.path.join(profiles_dir, f"{galaxy['Galaxy']}_surface_density.csv")
    if not os.path.isfile(profiles_filename):
        continue
    
    profiles_table = QTable.read(profiles_filename)
    theta = profiles_table['theta'] << u.arcsec
    surface_density = profiles_table['median'] << u.Msun/u.pc**2
    surface_density *= np.cos(inclination)
 
    # Mean surface density
    theta_bins = np.geomspace(theta[0]/2, theta[-1]*2, theta.size*10)
    theta_mid = (theta_bins[1:] + theta_bins[:-1]) / 2
    bin_area = np.pi * (theta_bins[1:]**2 - theta_bins[:-1]**2)
    bin_area[0] = np.pi * theta_bins[1]**2
    mean_surface_density = np.cumsum(bin_area * np.interp(theta_mid, theta, surface_density, right=0))
    mean_surface_density /= np.pi * theta_bins[1:]**2
                                     
    bins = np.where(SPARC_models['ID'] == galaxy['Galaxy'])
    SPARC_D = SPARC_models[bins]['D'][0]
    SPARC_R = SPARC_models[bins]['R']
    SPARC_theta = ((SPARC_models[bins]['R'] / SPARC_D).to_value(u.dimensionless_unscaled) << u.radian).to(u.arcsec)
    SPARC_obs = (SPARC_models[bins]['Vobs']**2 / SPARC_R / np.pi/c.G).to(u.Msun/u.pc**2)
    SPARC_gas = (SPARC_models[bins]['Vgas']**2 / SPARC_R / np.pi/c.G).to(u.Msun/u.pc**2)
    #SPARC_bul = (SPARC_models[bins]['Vbul']**2 / SPARC_R / np.pi/c.G).to(u.Msun/u.pc**2)
    #SPARC_disk = (SPARC_models[bins]['Vdisk']**2 / SPARC_R / np.pi/c.G).to(u.Msun/u.pc**2)

    mass_to_light_disk, mass_to_light_bul, SPARC_fit_D, SPARC_fit_inclination = np.loadtxt(
        os.path.join(input_dir, 'model_fits', 'ByGalaxy', 'Table', f'{galaxy["Galaxy"]}.mrt'),
        usecols=(1, 3, 5, 7), unpack=True)
    '''
    print('disk', np.nanpercentile(mass_to_light_disk, [16, 50, 84]))
    print('bul', np.nanpercentile(mass_to_light_bul, [16, 50, 84]))
    print('D', np.nanpercentile(SPARC_fit_D, [16, 50, 84]))
    print('i', np.nanpercentile(SPARC_fit_inclination, [16, 50, 84]))
    '''
    median_bul = np.nanmedian(mass_to_light_bul)
    median_disk = np.nanmedian(mass_to_light_disk)
    median_D = np.nanmedian(SPARC_fit_D)
    median_i = np.nanmedian(SPARC_fit_inclination)
    distance_rescaling = np.sqrt(median_D / SPARC_D)
    SPARC_bul = SPARC_models[bins]['SBbul'] * median_bul*u.Msun/u.Lsun * distance_rescaling
    SPARC_disk = SPARC_models[bins]['SBdisk'] * median_disk*u.Msun/u.Lsun * distance_rescaling
    #SPARC_stars = (median_bul*SPARC_bul + median_disk*SPARC_disk) * distance_rescaling
    SPARC_stars = SPARC_bul + SPARC_disk
    
    fig_name = f'{galaxy["Galaxy"]}_mass_model'
    fig, axes = new_figure(fig_name)
    ax = axes[0, 0]

    ax.plot(theta, surface_density, 'k-+', label=f'This work (i={inclination:.2f})')
    ax.plot(SPARC_theta, SPARC_stars, 'k:.', label=f'SPARC (i={median_i:.2f}, $\\Upsilon_{{bulge}}\sim${median_bul:.2f}, $\\Upsilon_{{disk}}\sim${median_disk:.2f})')
    '''
    ax.plot(theta, surface_density, 'k-+', label=f'$\\Sigma_\\star$ (i={inclination:.2f})')
    #ax.plot(theta_mid, mean_surface_density, 'k-', label=f'$\\bar\\Sigma_\\star$')
    #ax.plot(SPARC_theta, SPARC_obs*distance_rescaling, 'bo', label=f'SPARC $V^2/\pi GR$ (D={median_D:.2f})')
    ax.plot(SPARC_theta, SPARC_stars, 'b--', label=f'SPARC $\\bar\\Sigma_\\star$ (i={median_i:.2f})')
    ax.plot(SPARC_theta, SPARC_bul, 'r:', alpha=.5, label=f'$\\Upsilon_{{bulge}}\sim${median_bul:.2f}')
    ax.plot(SPARC_theta, SPARC_disk, 'b:', alpha=.5, label=f'$\\Upsilon_{{disk}}\sim${median_disk:.2f}')
    #ax.plot(SPARC_theta, median_bul*SPARC_bul*distance_rescaling, 'r:', alpha=.5, label=f'$\\Upsilon_{{bulge}}\sim${median_bul:.2f}')
    #ax.plot(SPARC_theta, median_disk*SPARC_disk*distance_rescaling, 'b:', alpha=.5, label=f'$\\Upsilon_{{disk}}\sim${median_disk:.2f}')
    #ax.plot(SPARC_theta, SPARC_gas*distance_rescaling, 'c--', alpha=.5, label='gas')
    ax.plot(SPARC_theta, SPARC_stars/distance_rescaling, 'b--', alpha=.2, label=f'SPARC fiducial D={SPARC_D:.2f}')
    '''

    ax.legend(title=f'{galaxy["Galaxy"]} (D={median_D:.2f} Mpc)')
    ax.set_yscale('log')
    ax.set_ylabel(r'surface density $\Sigma$ [M$_\odot$/pc$^2$]')
    ax.set_xlabel(r'galactocentric distance $\theta$ [arcsec]')
    
    fig.savefig(os.path.join(plots_dir, f"{fig_name}.png"), facecolor='white')
    if not show_plots:
        plt.close('all')
